# Análisis Global de Resultados – Clasificación de Defectos

Este notebook:
- Ejecuta el agregador final de métricas del pipeline
- Analiza resultados por dataset y ruta (FS vs NOFS)
- Prioriza métricas relevantes para **predicción / clasificación de defectos**
  (Recall, F1, MCC, BER)

In [161]:
# =========================================
# 1) SETUP + EJECUTAR AGREGADOR
# =========================================
import os
from pathlib import Path
from datetime import datetime

print("📂 Working directory ANTES:", os.getcwd())

# Si estamos dentro de notebooks/, subimos a la raíz del proyecto
if Path(os.getcwd()).name == "notebooks":
    os.chdir("..")

print("📂 Working directory AHORA:", os.getcwd())

# Ejecuta el agregador (genera/actualiza final_models_metrics.csv)
!python -m src.aggregate_final_metrics

📂 Working directory ANTES: /Users/behero/Documents/SCHOOL/UTM/MIS/2025/TERCER-SEMESTRE/prediccion-de-errores-en-la-ingenieria-de-software/JIRA-datasets-classification-pipeline/notebooks
📂 Working directory AHORA: /Users/behero/Documents/SCHOOL/UTM/MIS/2025/TERCER-SEMESTRE/prediccion-de-errores-en-la-ingenieria-de-software/JIRA-datasets-classification-pipeline
✅ Métricas agregadas correctamente.
   Archivo: artifacts/all_results/final_models_metrics.csv

Primeras filas:

    route balancing         dataset  ...  F1_Score Accuracy_sklearn  cv_best_score
282    fs  csbboost  activemq-5.0.0  ...  0.486111         0.803714       0.899316
284    fs  csbboost  activemq-5.0.0  ...  0.262673         0.151194       0.916088
283    fs  csbboost  activemq-5.0.0  ...  0.356923         0.445623       0.980193
114  nofs  csbboost  activemq-5.0.0  ...  0.558442         0.819629       0.908632
116  nofs  csbboost  activemq-5.0.0  ...  0.263279         0.153846       0.924088

[5 rows x 23 columns]


In [162]:
# =========================================
# 2) VERIFICAR ARCHIVOS GENERADOS
# =========================================
results_path = Path("artifacts/all_results")
print("📂 Carpeta de resultados (real):", results_path.resolve())

if not results_path.exists():
    raise FileNotFoundError(f"❌ No existe: {results_path}")

files = sorted(
    [f for f in results_path.iterdir() if f.is_file()],
    key=lambda x: x.stat().st_mtime,
    reverse=True
)

print("\n📄 Archivos generados en 'artifacts/all_results':")
for f in files:
    ts = datetime.fromtimestamp(f.stat().st_mtime).strftime("%Y-%m-%d %H:%M:%S")
    print(f"   ✅ {f.name}  |  {ts}")

📂 Carpeta de resultados (real): /Users/behero/Documents/SCHOOL/UTM/MIS/2025/TERCER-SEMESTRE/prediccion-de-errores-en-la-ingenieria-de-software/JIRA-datasets-classification-pipeline/artifacts/all_results

📄 Archivos generados en 'artifacts/all_results':
   ✅ final_models_metrics.csv  |  2025-12-12 03:25:07


In [163]:
# =========================================
# 3) CARGAR CSV FINAL + SANITY CHECKS
# =========================================
import numpy as np
import pandas as pd

possible_paths = [
    Path("artifacts/all_results/final_models_metrics.csv"),
    Path("../artifacts/all_results/final_models_metrics.csv"),
    Path("../../artifacts/all_results/final_models_metrics.csv"),
]

csv_path = next((p for p in possible_paths if p.exists()), None)
if csv_path is None:
    raise FileNotFoundError("❌ No se encontró final_models_metrics.csv")

print("📄 Usando archivo:", csv_path)
ts = csv_path.stat().st_mtime
print("🕒 CSV last modified:", datetime.fromtimestamp(ts).isoformat(timespec="seconds"))

df = pd.read_csv(csv_path)

required_cols = {"dataset", "route", "balancing", "scaler", "model", "F1_Score", "Recall", "Precision"}
missing = required_cols - set(df.columns)
if missing:
    raise ValueError(f"❌ Faltan columnas requeridas: {missing}")

print("✅ Total de filas:", len(df))
print("✅ Rutas:", df["route"].value_counts().to_dict())

display(df.head())
df.info()

📄 Usando archivo: artifacts/all_results/final_models_metrics.csv
🕒 CSV last modified: 2025-12-12T03:25:07
✅ Total de filas: 336
✅ Rutas: {'fs': 168, 'nofs': 168}


,route,balancing,dataset,scaler,model,TP,TN,FP,FN,P,...,Exactitud,TP_Rate,FP_Rate,Precision,Recall,Sensibilidad,Especificidad,F1_Score,Accuracy_sklearn,cv_best_score
0,fs,csbboost,activemq-5.0.0,robust,decision_tree,35,268,50,24,59,...,0.803714,0.593220,0.157233,0.411765,0.593220,0.593220,0.842767,0.486111,0.803714,0.899316
1,fs,csbboost,activemq-5.0.0,robust,naive_bayes_gaussian,57,0,318,2,59,...,0.151194,0.966102,1.000000,0.152000,0.966102,0.966102,0.000000,0.262673,0.151194,0.916088
2,fs,csbboost,activemq-5.0.0,robust,svm,58,110,208,1,59,...,0.445623,0.983051,0.654088,0.218045,0.983051,0.983051,0.345912,0.356923,0.445623,0.980193
3,nofs,csbboost,activemq-5.0.0,robust,decision_tree,43,266,52,16,59,...,0.819629,0.728814,0.163522,0.452632,0.728814,0.728814,0.836478,0.558442,0.819629,0.908632
4,nofs,csbboost,activemq-5.0.0,robust,naive_bayes_gaussian,57,1,317,2,59,...,0.153846,0.966102,0.996855,0.152406,0.966102,0.966102,0.003145,0.263279,0.153846,0.924088


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 336 entries, 0 to 335
Data columns (total 23 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   route             336 non-null    object 
 1   balancing         336 non-null    object 
 2   dataset           336 non-null    object 
 3   scaler            336 non-null    object 
 4   model             336 non-null    object 
 5   TP                336 non-null    int64  
 6   TN                336 non-null    int64  
 7   FP                336 non-null    int64  
 8   FN                336 non-null    int64  
 9   P                 336 non-null    int64  
 10  N                 336 non-null    int64  
 11  Total             336 non-null    int64  
 12  Error             336 non-null    float64
 13  Exactitud         336 non-null    float64
 14  TP_Rate           336 non-null    float64
 15  FP_Rate           336 non-null    float64
 16  Precision         336 non-null    float64
 1

In [164]:
# =========================================
# 4) ASEGURAR MCC Y BER (desde TP/TN/FP/FN)
# =========================================
has_conf = {"TP", "TN", "FP", "FN"}.issubset(df.columns)

def safe_mcc(tp, tn, fp, fn):
    num = (tp * tn) - (fp * fn)
    den = np.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))
    return np.where(den == 0, np.nan, num / den)

def safe_ber(tp, tn, fp, fn):
    fnr = np.where((tp + fn) == 0, np.nan, fn / (tp + fn))
    fpr = np.where((tn + fp) == 0, np.nan, fp / (tn + fp))
    return 0.5 * (fnr + fpr)

if "MCC" not in df.columns:
    df["MCC"] = safe_mcc(df["TP"], df["TN"], df["FP"], df["FN"]) if has_conf else np.nan

if "BER" not in df.columns:
    df["BER"] = safe_ber(df["TP"], df["TN"], df["FP"], df["FN"]) if has_conf else np.nan

In [165]:
# =========================================
# 5) FILTRADO DE RESULTADOS DEGENERADOS
# =========================================
df_filtered = df.copy()

# Evita modelos colapsados por especificidad
if "Especificidad" in df_filtered.columns:
    df_filtered = df_filtered[df_filtered["Especificidad"] > 0]
elif "TN" in df_filtered.columns:
    df_filtered = df_filtered[df_filtered["TN"] > 0]

# Evita colapso "todo No-Buggy" => TP=0 => Recall=0
if "Recall" in df_filtered.columns:
    df_filtered = df_filtered[df_filtered["Recall"] > 0]
elif "TP" in df_filtered.columns:
    df_filtered = df_filtered[df_filtered["TP"] > 0]

print("✅ Filas tras filtrado:", len(df_filtered))

✅ Filas tras filtrado: 320


In [166]:
# =========================================
# 6) BEST POR DATASET Y RUTA (FS vs NOFS)
# =========================================
sort_cols = ["F1_Score", "Recall", "Precision", "MCC"]
ascending = [False, False, False, False]

df_filtered["_neg_BER"] = -df_filtered["BER"]
sort_cols.append("_neg_BER")
ascending.append(False)

best_per_dataset = (
    df_filtered
    .sort_values(sort_cols, ascending=ascending)
    .groupby(["dataset", "route"], as_index=False)
    .first()
    .drop(columns="_neg_BER")
)

best_table = best_per_dataset[
    [
        "dataset", "route", "balancing", "scaler", "model",
        "TP", "TN", "FP", "FN",
        "F1_Score", "Recall", "Precision", "MCC", "BER",
        "Error", "Accuracy_sklearn", "cv_best_score"
    ]
].copy()

for c in ["F1_Score", "Recall", "Precision", "MCC", "BER", "Error", "Accuracy_sklearn", "cv_best_score"]:
    if c in best_table.columns:
        best_table[c] = pd.to_numeric(best_table[c], errors="coerce").round(2)

display(best_table.sort_values(["dataset", "route"]))

,dataset,route,balancing,scaler,model,TP,TN,FP,FN,F1_Score,Recall,Precision,MCC,BER,Error,Accuracy_sklearn,cv_best_score
0,activemq-5.0.0,fs,unbalanced,robust,decision_tree,35,296,22,24,0.60,0.59,0.61,0.53,0.24,0.12,0.88,0.58
1,activemq-5.0.0,nofs,unbalanced,robust,decision_tree,40,293,25,19,0.65,0.68,0.62,0.58,0.20,0.12,0.88,0.62
2,derby-10.5.1.1,fs,csbboost,standard,naive_bayes_gaussian,30,429,35,47,0.42,0.39,0.46,0.34,0.34,0.15,0.85,0.38
3,derby-10.5.1.1,nofs,hcbou,standard,naive_bayes_gaussian,35,422,42,42,0.45,0.45,0.45,0.36,0.32,0.16,0.84,0.46
4,groovy-1_6_BETA_1,fs,hcbou,robust,svm,10,137,14,4,0.53,0.71,0.42,0.49,0.19,0.11,0.89,0.98
5,groovy-1_6_BETA_1,nofs,hcbou,standard,svm,7,137,14,7,0.40,0.50,0.33,0.34,0.30,0.13,0.87,0.97
6,hbase-0.94.0,fs,hcbou,standard,decision_tree,34,134,34,10,0.61,0.77,0.50,0.50,0.21,0.21,0.79,0.85
7,hbase-0.94.0,nofs,csbboost,robust,decision_tree,33,148,20,11,0.68,0.75,0.62,0.59,0.18,0.15,0.85,0.86
8,hive-0.9.0,fs,unbalanced,robust,decision_tree,36,206,21,21,0.63,0.63,0.63,0.54,0.23,0.15,0.85,0.60
9,hive-0.9.0,nofs,csbboost,robust,decision_tree,48,189,38,9,0.67,0.84,0.56,0.59,0.16,0.17,0.83,0.87


In [167]:
# =========================================
# 7) TABLA FINAL – SOLO MÉTRICAS RECOMENDADAS
# =========================================
best_metrics = best_per_dataset.copy()

for c in ["TP", "TN", "FP", "FN", "Recall", "Precision", "F1_Score", "MCC", "BER"]:
    if c in best_metrics.columns:
        best_metrics[c] = pd.to_numeric(best_metrics[c], errors="coerce")

eps = 1e-12
best_metrics["Specificity"] = best_metrics["TN"] / (best_metrics["TN"] + best_metrics["FP"] + eps)
best_metrics["BalancedAcc"] = 1.0 - best_metrics["BER"]  # si BER = 0.5(FNR+FPR)

recommended_cols = [
    "dataset", "route", "model",
    "Recall", "Precision", "F1_Score", "MCC", "BER",
    "BalancedAcc", "Specificity",
]

final_metrics_table = best_metrics[recommended_cols].copy()

for c in ["Recall", "Precision", "F1_Score", "MCC", "BER", "BalancedAcc", "Specificity"]:
    final_metrics_table[c] = final_metrics_table[c].round(3)

final_metrics_table = final_metrics_table.sort_values(
    by=["dataset", "route", "F1_Score", "Recall", "MCC", "BER"],
    ascending=[True, True, False, False, False, True],
)

display(final_metrics_table)

,dataset,route,model,Recall,Precision,F1_Score,MCC,BER,BalancedAcc,Specificity
0,activemq-5.0.0,fs,decision_tree,0.593,0.614,0.603,0.531,0.238,0.762,0.931
1,activemq-5.0.0,nofs,decision_tree,0.678,0.615,0.645,0.576,0.200,0.800,0.921
2,derby-10.5.1.1,fs,naive_bayes_gaussian,0.390,0.462,0.423,0.338,0.343,0.657,0.925
3,derby-10.5.1.1,nofs,naive_bayes_gaussian,0.455,0.455,0.455,0.364,0.318,0.682,0.909
4,groovy-1_6_BETA_1,fs,svm,0.714,0.417,0.526,0.491,0.189,0.811,0.907
5,groovy-1_6_BETA_1,nofs,svm,0.500,0.333,0.400,0.341,0.296,0.704,0.907
6,hbase-0.94.0,fs,decision_tree,0.773,0.500,0.607,0.496,0.215,0.785,0.798
7,hbase-0.94.0,nofs,decision_tree,0.750,0.623,0.680,0.591,0.185,0.815,0.881
8,hive-0.9.0,fs,decision_tree,0.632,0.632,0.632,0.539,0.230,0.770,0.907
9,hive-0.9.0,nofs,decision_tree,0.842,0.558,0.671,0.588,0.163,0.837,0.833


In [168]:
# =========================================
# 8) FS vs NOFS (Δ métricas)
# =========================================
routes_per_ds = best_per_dataset.groupby("dataset")["route"].nunique().sort_values()
missing_route_datasets = routes_per_ds[routes_per_ds < 2].index.tolist()

if missing_route_datasets:
    print("⚠️ Estos datasets NO tienen ambas rutas (fs y nofs). Deltas tendrán NaN:")
    print("   ", missing_route_datasets)
else:
    print("✅ Todos los datasets tienen ambas rutas (fs y nofs).")

comparison = best_per_dataset.pivot(
    index="dataset",
    columns="route",
    values=["F1_Score", "Recall", "Precision", "MCC", "BER"]
)

def _safe_delta(metric_name: str):
    fs = comparison.get((metric_name, "fs"))
    nofs = comparison.get((metric_name, "nofs"))
    if fs is None or nofs is None:
        return np.nan
    return fs - nofs

comparison[("Δ_F1", "")] = _safe_delta("F1_Score")
comparison[("Δ_Recall", "")] = _safe_delta("Recall")
comparison[("Δ_MCC", "")] = _safe_delta("MCC")
comparison[("Δ_BER", "")] = _safe_delta("BER")  # negativo = mejora (BER menor es mejor)

display(comparison.sort_values(("Δ_F1", ""), ascending=False))

delta_summary = comparison[
    [("Δ_F1", ""), ("Δ_Recall", ""), ("Δ_MCC", ""), ("Δ_BER", "")]
].agg(["mean", "median", "min", "max"]).round(2)

display(delta_summary)

✅ Todos los datasets tienen ambas rutas (fs y nofs).


F1_Score              Recall           Precision  \
route                     fs      nofs        fs      nofs        fs   
dataset                                                                
groovy-1_6_BETA_1   0.526316  0.400000  0.714286  0.500000  0.416667   
jruby-1.1           0.666667  0.647059  0.588235  0.647059  0.769231   
wicket-1.3.0-beta2  0.343750  0.343750  0.423077  0.423077  0.289474   
derby-10.5.1.1      0.422535  0.454545  0.389610  0.454545  0.461538   
hive-0.9.0          0.631579  0.671329  0.631579  0.842105  0.631579   
activemq-5.0.0      0.603448  0.645161  0.593220  0.677966  0.614035   
hbase-0.94.0        0.607143  0.680412  0.772727  0.750000  0.500000   

                                   MCC                 BER            \
route                   nofs        fs      nofs        fs      nofs   
dataset                                                                
groovy-1_6_BETA_1   0.333333  0.491278  0.340533  0.189215  0.296358   
jruby-1.1           0.647059  0.636564  0.600905  0.217421  0.199548   
wicket-1.3.0-beta2  0.289474  0.286973  0.286973  0.329746  0.329746   
derby-10.5.1.1      0.454545  0.337615  0.364028  0.342910  0.317986   
hive-0.9.0          0.558140  0.539068  0.588142  0.230466  0.162648   
activemq-5.0.0      0.615385  0.531484  0.576482  0.237981  0.200325   
hbase-0.94.0        0.622642  0.495546  0.590937  0.214827  0.184524   

                        Δ_F1  Δ_Recall     Δ_MCC     Δ_BER  
route                                                       
dataset                                                     
groovy-1_6_BETA_1   0.126316  0.214286  0.150745 -0.107143  
jruby-1.1           0.019608 -0.058824  0.035659  0.017873  
wicket-1.3.0-beta2  0.000000  0.000000  0.000000  0.000000  
derby-10.5.1.1     -0.032010 -0.064935 -0.026413  0.024924  
hive-0.9.0         -0.039750 -0.210526 -0.049074  0.067818  
activemq-5.0.0     -0.041713 -0.084746 -0.044999  0.037656  
hbase-0.94.0       -0.073270  0.022727 -0.095391  0.030303

,Δ_F1,Δ_Recall,Δ_MCC,Δ_BER
route,,,,
mean,-0.01,-0.03,-0.00,0.01
median,-0.03,-0.06,-0.03,0.02
min,-0.07,-0.21,-0.10,-0.11
max,0.13,0.21,0.15,0.07


In [169]:
!pwd
os.chdir("notebooks")
print(os.getcwd())

/Users/behero/Documents/SCHOOL/UTM/MIS/2025/TERCER-SEMESTRE/prediccion-de-errores-en-la-ingenieria-de-software/JIRA-datasets-classification-pipeline
/Users/behero/Documents/SCHOOL/UTM/MIS/2025/TERCER-SEMESTRE/prediccion-de-errores-en-la-ingenieria-de-software/JIRA-datasets-classification-pipeline/notebooks


In [171]:
!jupyter nbconvert explore_final_results.ipynb --to html

[NbConvertApp] Converting notebook explore_final_results.ipynb to html
[NbConvertApp] Writing 334906 bytes to explore_final_results.html
